In [2]:
import os
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
### all the determined variables
main_dir = "../GameObservers/"
sub_dir = "/DartBoard/"
result_dir = "../graphs/histogram/"
control = set(['ah', 'aj', 'dd', 'dl', 'ez', 'it', 'll', 'sh', 'sm', 'sr']) #10 stereo-normal
exper = set(['bb', 'by', 'co', 'et', 'gn', 'gp', 'jz', 'kp', 'ky', 'mb', 'mg', 'ni', 'tp']) #13 stereo-anomalous

In [4]:
def get_vals(obs_set, group):
    dfs = []
    for obs in obs_set:
        path = main_dir + obs + sub_dir
        for data in os.listdir(path):
            if not data.endswith('.csv'):
                continue
            df = pd.read_csv(path + data)  
            df = df.copy()
            df['Group'] = group
            dfs.append(df)
    data = pd.concat(dfs)
    data.rename(columns={'SA[seconds] dartboard hit':'StereoAcuity'}, inplace=True)
    return data

In [5]:
def number2String(N):
    if N == 1:
        return "natural"
    if N == 2:
        return "advanced"
    if N == 3:
        return "expert"

In [6]:
control_df = get_vals(control, "Control")
exper_df = get_vals(exper, "Experimental")
aggregate_df = pd.concat([control_df, exper_df])
    
aggregate_df["Difficulty_String"] = [number2String(element) for element in aggregate_df.Difficulty]

C:\Users\angie\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [7]:
ols_lm = smf.ols('StereoAcuity ~ Difficulty_String*Group',data=aggregate_df)

fit = ols_lm.fit()

table = sm.stats.anova_lm(fit, typ=2)

table

,sum_sq,df,F,PR(>F)
Difficulty_String,3.852386e+07,2.0,409.321997,5.720305e-178
Group,2.141848e+08,1.0,4551.493243,0.000000e+00
Difficulty_String:Group,1.685433e+07,2.0,179.079823,2.123659e-78
Residual,6.512611e+09,138395.0,NaN,NaN
